# 1. 运行示例 Planner
本项目要求你实现自己的 Planner 来对一系列由语言描述的具身智能任务进行规划。

在介绍 Planner 接口之前，让我们先尝试运行一个简单的 LLM Planner，其实现位于 [python_package/embodiment/planner.py](./python_package/embodiment/planner.py)，你可以直接修改该 Planner 来实现你自己的 Planner。

运行本教程前请按照 [README](./README.md) 中的说明正确安装环境，并将数据存放在当前目录下的 `data` 文件夹中。


In [ ]:
import os
# 命令行环境下，以下环境变量通过 `export OPENAI_API_KEY=xxx` 设置
# 或者将其写在 `env.bash`，然后在执行脚本前执行 `source env.bash`
# Jupyter Notebook 环境下，以下环境变量通过 `os.environ['OPENAI_API_KEY'] = 'xxx'` 设置
os.environ['OPENAI_API_KEY'] = 'EMPTY'
os.environ['OPENAI_ENDPOINT'] = 'http://localhost:8000/v1'
os.environ['OPENAI_MODEL_NAME'] = 'Qwen/Qwen3-8B'

本文档使用 [conf/example_simple_planner.yaml](./conf/example_simple_planner.yaml) 中的配置初始化环境，并交互式的执行特定 episode。如果你想在验证集上测评该 Planner 的效果，可以通过以下指令

```bash
python scripts/evaluation.py --config-name example_simple_planner
```

## 1.1 初始化实验环境

以下代码帮助你初始化整个实验环境，包括

- 加载 dataset
- 初始化仿真环境
- 初始化 world graph
- 初始化 planning agents
- 初始化 planner

测评脚本将会以同样的方式初始化实验环境。



In [ ]:
from hydra import initialize, compose
from python_package.embodiment.runner import PartnerRunner
initialize(config_path="conf")
config = compose(config_name="example_simple_planner",
                 overrides=["evaluation.output_dir=outputs", "paths.results_dir=outputs"])

runner = PartnerRunner(config)

PluginManager::Manager: duplicate static plugin StbImageImporter, ignoring
PluginManager::Manager: duplicate static plugin GltfImporter, ignoring
PluginManager::Manager: duplicate static plugin BasisImporter, ignoring
PluginManager::Manager: duplicate static plugin AssimpImporter, ignoring
PluginManager::Manager: duplicate static plugin AnySceneImporter, ignoring
PluginManager::Manager: duplicate static plugin AnyImageImporter, ignoring
/tmp/ipykernel_3489509/531476864.py:3: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path="conf")
/home/yongxi/App/anaconda3/envs/habitat-llm/lib/python3.9/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'example_simple_planner': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
2025-05-20 19:39:3

Finished setting up config


PluginManager::Manager: duplicate static plugin StbImageImporter, ignoring
PluginManager::Manager: duplicate static plugin GltfImporter, ignoring
PluginManager::Manager: duplicate static plugin BasisImporter, ignoring
PluginManager::Manager: duplicate static plugin AssimpImporter, ignoring
PluginManager::Manager: duplicate static plugin AnySceneImporter, ignoring
PluginManager::Manager: duplicate static plugin AnyImageImporter, ignoring
2025-05-20 19:39:39,431 Initializing task RearrangeEmptyTask-v0
MeshTools::compile(): ignoring unknown/unsupported attribute Trade::MeshAttribute::Custom(0)
MeshTools::compile(): ignoring unknown/unsupported attribute Trade::MeshAttribute::Custom(1)


successfully added agent with UID : 0
finished initializing agents!


## 1.2 读取数据集信息

本题目使用 partnr benchmark 提供的 `CollaborationDatasetV0` 数据集格式，每个数据集包含多个 Episode `CollaborationEpisode`，每个 Episode 即为一个任务对象。

算法测评过程中会遍历数据集中每个 Episode 并调用 Planner 进行规划和执行，你可以通过以下方式获取 Episode 信息。

In [3]:
from habitat_llm.agent.env.dataset import CollaborationEpisode
episode_id: str = runner.current_episode_id
current_episode: CollaborationEpisode = [e for e in runner.episodes if e.episode_id == episode_id][0]
print(f"当前 episode\nid:\t\t{current_episode.episode_id}\ninstructions:\t{current_episode.instruction}")

当前 episode
id:		1672
instructions:	Move the kettle, jug, and teapot from the dining table to the kitchen, fill the kettle with water and turn it on. Then, clean the dining table.


每个 Episode 都定义了其自身的 Evaluation Functions，你可以参考 [Partnr Doc: Habitat-LLM Evaluation Engine](https://github.com/RoboticSJTU/partnr-planner/blob/sii/habitat_llm/agent/env/evaluation/README.md) 来了解 episode 执行成功的具体判定条件，以及 [API Doc: CollaborationEpisode](http://localhost:8000/api/CollaborationEpisode/) 来了解如何在调试过程中获取这些信息。

## 1.3 执行 Episode

以下代码初始化仿真环境并调用 Planner，你不需要关心环境初始化过程，测评过程中有测评代码负责仿真环境初始化。

In [ ]:
# 注意 episode_id 的类型必须是 str，而不能是 int
runner.reset(episode_id=episode_id)
info = runner.run_instruction(current_episode.instruction, output_name="test")

Instruction:
Move the kettle, jug, and teapot from the dining table to the kitchen, fill the kettle with water and turn it on. Then, clean the dining table.

{0: ('Navigate', 'table_25', None)}
Thought: First, I need to locate the kettle, jug, and teapot on the dining table. Since the dining table is in the dining_room_1, I will navigate to the table_25 in dining_room_1 to find these objects.
Navigate[table_25]
{0: 'Successful execution!'}
{0: ('Done', None, None)}
Thought: First, I need to pick up the kettle from the dining table.
Pick[kettle_0]

Thought: Now that I have the kettle, I should navigate to the kitchen to place it there.
Navigate[kitchen_1]

Thought: I need to place the kettle on a suitable receptacle in the kitchen. The counter has multiple receptacles available.
Place[kettle_0, on, counter_47, None, None]

Thought: Next, I should go back to the dining table to pick up the jug.
Navigate[dining_room_1]

Thought: Now, I will pick up the jug from the dining table.
Pick[jug_

你可以在 `outputs/validation_episodes.json.gz` 路径下看到执行的输出，在其 `videos` 子目录下看到渲染出的视频。由于当前 Planner 实现较为简易，可能无法正常完成任务。

到目前为止，你已经可以调用示例 Planner 进行任务规划并查看结果，接下来请参照 [tut02_implement_your_own_planner](./tut02_implement_your_own_planner.ipynb) 来实现新的 Planner。